# Maximum flow problem

Find the path with the maximum flow (throughput) from node `s` to node `d`. I'm going to assume `s = 1`, `d = last node (12)`.

First, let's introduce the types for representing the graph: lists of `vertices` (each being a tuple of _vertex number_ and _list of vertices merged into it_, more on this later) and `edges` (each being a tuple of _source vertex_, _destination vertex_, and _list of weights (throughputs) of vertices merged into the source_).

In [1]:
type V = [(Int, [Int])]
type E = [(Int, Int, [Int])]

Next, let's define the givens. The input is a vertex matrix, but it's easier to manipulate the graph as vertices and edges separately, so I'm going to transform it:

In [2]:
vertexMatrix = [ [0, 3, 5, 3, 0, 1, 2, 1, 0, 0, 3, 5 ]
               , [3, 0, 0, 5, 1, 4, 2, 0, 0, 0, 4, 0 ]
               , [5, 0, 0, 0, 0, 5, 1, 1, 3, 0, 5, 5 ]
               , [3, 5, 0, 0, 5, 0, 1, 0, 0, 2, 0, 0 ]
               , [0, 1, 0, 5, 0, 4, 0, 2, 5, 4, 0, 3 ]
               , [1, 4, 5, 0, 4, 0, 0, 4, 5, 0, 0, 0 ]
               , [2, 2, 1, 1, 0, 0, 0, 4, 0, 2, 0, 0 ]
               , [1, 0, 1, 0, 2, 4, 4, 0, 1, 0, 4, 4 ]
               , [0, 0, 3, 0, 5, 5, 0, 1, 0, 0, 5, 0 ]
               , [0, 0, 0, 2, 4, 0, 2, 0, 0, 0, 0, 3 ]
               , [3, 4, 5, 0, 0, 0, 0, 4, 5, 0, 0, 0 ]
               , [5, 0, 5, 0, 3, 0, 0, 4, 0, 3, 0, 0 ] ]

vertices = [ (n, [n]) | n <- [0..(length vertexMatrix - 1)] ]
edges = [ (s, d, [w]) | s <- [0..11], d <- [0..11],
                        let w = (vertexMatrix !! s !! d),
                        s < d, w /= 0 ]

Now, we're getting to the algorithm itself. The first step is making a cut at the `s` node and finding the maximum throughput of edges at it: 

In [3]:
maxFlowInCut :: Int -> E -> Int
maxFlowInCut s es =
  maximum [ w | (s', _, ws) <- es, w <- ws, s' == s ]

Edges having throughput more or equal to `maxFlow` are selected, and _destination vertices_ of them are merged into _source vertices_. It's for this reason we keep a list of merged vertices in every vertex tuple.

In [4]:
import Data.List ((\\), find, break, partition, deleteBy, sort)

candidateEdges :: E -> Int -> [(Int, Int)]
candidateEdges es maxf =
  reverse [ (s, d) | (s, d, ws) <- es, maximum ws >= maxf ]

mergeCandidates :: [(Int, Int)] -> (V, E) -> (V, E)
mergeCandidates candidates graph = go graph candidates
  where
    go (vs, es) ((to, from):cs) =
      let [to', from'] = sort [resolveVertex vs to, resolveVertex vs from]
          vs' = mergeTwoVertices to' from' vs
          es' = mergeTwoEdges to' from' es
      in go (vs', es') cs
    go graph [] = graph
    resolveVertex vs v
      | Just (v', _) <- find ((v `elem`) . snd) vs = v'
      | otherwise = v

mergeTwoVertices :: Int -> Int -> V -> V
mergeTwoVertices to from vertices = case vs of
    [] -> vertices
    ((_, toVertices):vstail) -> (vsbefore ++ (to, toVertices ++ fromVertices) : vstail) 
  where
    (vsbefore, vs) = break ((to ==) . fst) vertices'
    (vertices', fromVertices) = case break ((from ==) . fst) vertices of
      (vs, (_, fromVertices):vs') -> (vs ++ vs', fromVertices)
      (vs, []) -> (vs, [])

mergeTwoEdges :: Int -> Int -> E -> E
mergeTwoEdges to from edges = go touched untouched
  where
    go (t:ts) es = go ts (merge es t)
    go [] es = es
    (touched, untouched) = partition shouldUpdate edges'
    shouldUpdate (s, d, _) = s == from || d == from
    edges' = filter (not . shouldDrop) edges
    shouldDrop (s, d, _) = s == to && d == from
    merge es (s, d, w)
      | s == from
      = case (partition (\(s', d', _) -> (s', d') == (to, d) || (s', d') == (d, to)) es) of
          ([(_, _, w')], es') -> (edge to d (w ++ w')) : es'
          ([], es') -> (edge to d w) : es'
      | d == from
      = case partition (\(s', d', _) -> (s', d') == (s, to) || (s', d') == (to, s)) es of
          ([(_, _, w')], es') -> (edge s to (w ++ w')) : es'
          ([], es') -> (edge s to w) : es'
    edge to from w = (minimum [to, from], maximum [to, from], w)

All that remains is properly displaying the result and intermediate steps. `mergeSD` is the entry point of algorithm, which iterates over the graph (making a cut, finding candidate edges, and merging vertices) __until__ `s` and `d` are merged into a single vertex.

In [5]:
import Control.Monad.Writer
import IHaskell.Display (plain)

-- (max flow, candidates, graph before, graph after)
type IterLog = (Int, [(Int, Int)], (V, E), (V, E))

mergeSD :: Int -> Int -> (V, E) -> Writer [IterLog] (V, E)
mergeSD s d = go
  where
    go (vs, es)
      | areMerged s d vs = return (vs, es)
      | otherwise = iter (vs, es) >>= go
    iter :: (V, E) -> Writer [IterLog] (V, E)  
    iter (vs, es) = do
      let graph' = mergeCandidates candidates (vs, es)
          candidates = candidateEdges es maxFlow
          maxFlow = maxFlowInCut s es
      tell $ pure (maxFlow, candidates, (vs, es), graph')
      return graph'
    areMerged v1 v2 = any (\(v, vs) -> v == v1 && v2 `elem` vs)

Graph rendering is handled via GraphViz:

In [6]:
:set -XScopedTypeVariables

import Data.Graph.Inductive (Gr(..), mkGraph)
import qualified Data.GraphViz as GV
import qualified Data.GraphViz.Attributes.Complete as GA
import System.IO.Strict (hGetContents)
import Data.List (intercalate)
import IHaskell.Display (DisplayData)

displayGraph :: (V, E) -> IO DisplayData
displayGraph = (((IHaskell.Display.html <$>) . renderSvg) .) $ uncurry graph
  where
    graph :: V -> E -> Gr [Int] [Int] = mkGraph
    renderSvg g = GV.graphvizWithHandle GV.Dot (GV.graphToDot params g) GV.Svg hGetContents
    params = GV.nonClusteredParams { GV.fmtNode = \(n, a) -> [GV.toLabel $ joinIds . ((+1) <$>) $ a],
                                     GV.fmtEdge = \(_,_,l) -> [GV.toLabel $ joinIds l],
                                     GV.globalAttributes = [GV.GraphAttrs [GA.RankDir GA.FromLeft]] }
    joinIds :: [Int] -> String = intercalate ", " . (show <$>)

Finally, we print some basic information about each intermediate step, and render the graph obtained as its result.

In [7]:
printLog :: IterLog -> IO [DisplayData]
printLog (maxFlow, candidates, (vs, es), graph') = do
  let h = plain <$> ["Qmax = " ++ show maxFlow, showCandidates]
  graph <- displayGraph graph'
  return (h ++ [graph])
  where
    showCandidates = "Candidates: " ++ intercalate ", " (foldl showCandidate [] candidates)
    showCandidate :: [String] -> (Int, Int) -> [String]
    showCandidate acc (s, d)
      | Just (_, svs) <- find ((s ==) . fst) vs,
        Just (_, dvs) <- find ((d ==) . fst) vs
      = (showVertices (svs ++ dvs) : acc)
    showVertices :: [Int] -> String
    showVertices vs' = "(" ++ intercalate ", " (show . (+ 1) <$> vs') ++ ")"

In [8]:
let s = 0
let d = fst $ last vertices

let (_, ils) = runWriter (mergeSD s d (vertices, edges))

printLog <$> ils

let (maxFlow, _, _, _) = last ils

maxFlowEdges = [ (s, d, w) | (s, d, w) <- edges, maximum w >= maxFlow ]
plain "Maximum throughput graph:"
displayGraph (vertices, maxFlowEdges)

Qmax = 5

Candidates: (1, 3), (1, 12), (2, 4), (3, 6), (3, 11), (3, 12), (4, 5), (5, 9), (6, 9), (9, 11)

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 2.40.1 (20161225.0304)
 -->
<!-- Title: %3 Pages: 1 -->
 
 
 %3 
 
<!-- 0 -->
 
 0 
 
 1, 2, 3, 12, 4, 5, 6, 9, 11 
 
<!-- 6 -->
 
 6 
 
 7 
 
<!-- 0->6 -->
 
 0->6 
 
 
 1, 1, 2, 2 
 
<!-- 7 -->
 
 7 
 
 8 
 
<!-- 0->7 -->
 
 0->7 
 
 
 4, 1, 4, 2, 4, 1, 1 
 
<!-- 9 -->
 
 9 
 
 10 
 
<!-- 0->9 -->
 
 0->9 
 
 
 4, 2, 3 
 
<!-- 6->7 -->
 
 6->7 
 
 
 4 
 
<!-- 6->9 -->
 
 6->9 
 
 
 2

Maximum throughput graph:

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 2.40.1 (20161225.0304)
 -->
<!-- Title: %3 Pages: 1 -->
 
 
 %3 
 
<!-- 0 -->
 
 0 
 
 1 
 
<!-- 2 -->
 
 2 
 
 3 
 
<!-- 0->2 -->
 
 0->2 
 
 
 5 
 
<!-- 11 -->
 
 11 
 
 12 
 
<!-- 0->11 -->
 
 0->11 
 
 
 5 
 
<!-- 1 -->
 
 1 
 
 2 
 
<!-- 3 -->
 
 3 
 
 4 
 
<!-- 1->3 -->
 
 1->3 
 
 
 5 
 
<!-- 5 -->
 
 5 
 
 6 
 
<!-- 2->5 -->
 
 2->5 
 
 
 5 
 
<!-- 10 -->
 
 10 
 
 11 
 
<!-- 2->10 -->
 
 2->10 
 
 
 5 
 
<!-- 2->11 -->
 
 2->11 
 
 
 5 
 
<!-- 4 -->
 
 4 
 
 5 
 
<!-- 3->4 -->
 
 3->4 
 
 
 5 
 
<!-- 8 -->
 
 8 
 
 9 
 
<!-- 4->8 -->
 
 4->8 
 
 
 5 
 
<!-- 5->8 -->
 
 5->8 
 
 
 5 
 
<!-- 6 -->
 
 6 
 
 7 
 
<!-- 7 -->
 
 7 
 
 8 
 
<!-- 8->10 -->
 
 8->10 
 
 
 5 
 
<!-- 9 -->
 
 9 
 
 10